#Set up

In [22]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

SERPER_API_KEY = userdata.get('SERPER_API_KEY')
os.environ['SERPER_API_KEY'] = SERPER_API_KEY

In [23]:
!pip install crewai openai

In [24]:
from crewai import Agent, Task, Crew, LLM

# Initialize Large Language Model (LLM) of your choice (see all models on our Models page)
llm = LLM(model="openai/gpt-3.5-turbo")

#crewAI Tools

In [25]:
!pip install crewai-tools

In [26]:
import warnings
warnings.filterwarnings('ignore')

from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')


#Creating Agents

In [27]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    llm=llm,
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [28]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    #tools = [scrape_tool, search_tool,read_resume, semantic_search_resume],
    llm=llm,
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [29]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    #tools = [scrape_tool, search_tool,read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [30]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    #tools = [scrape_tool, search_tool,read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

#Creating Tasks

In [31]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [32]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [33]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [34]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

#Creating the Crew

In [35]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

#Running the Crew

In [36]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [37]:

### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 91456039-e4fc-440a-b5f3-40880b45ab1d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: ca1685aa-4d5c-4111-976f-aea009b17d22
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: ca1685aa-4d5c-4111-976f-aea009b17d22
│      Status: Executing Task...
└── 📋 Task: 8c3cc51c-f38a-4977-aa40-bcbe389717b6
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: ca1685aa-4d5c-4111-976f-aea009b17d22
│      Status: Executing Task...
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
└── 📋 Task: 8c3cc51c-f38a-4977-aa40-bcbe389717b6
       Status: Executing Task...

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.


...



# Agent: Engineering Interview Preparer
## Final Answer: 
**Interview Questions and Talking Points for Noah**  

1. **Questions about Professional Experience:**
   - Can you elaborate on the key technology initiatives you led, and how they improved operational efficiency by 30%?  
     *Talking Point: Discuss specifics of the project, including team dynamics, challenges faced, and innovative strategies employed to achieve the results. Highlight how you fostered collaboration in a remote environment, which is crucial for the AI Fund's work culture.*

2. **Startup Leadership:**
   - What are some of the biggest challenges you faced while founding and scaling startup ventures, and how did you overcome them?  
     *Talking Point: Illustrate your entrepreneurial spirit, strategic planning, and ability to adapt in a fast-paced environment. Provide examples of how your leadership style contributed to the startups' growth and success.*

3. **Team Management:**
   - How do you approach team-

🚀 Crew: crew
├── 📋 Task: ca1685aa-4d5c-4111-976f-aea009b17d22
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: 8c3cc51c-f38a-4977-aa40-bcbe389717b6
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (2)
│       └── 🔧 Used Read website content (2)
├── 📋 Task: d8b2bdeb-4a3a-48a3-9493-d8d194206740
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
└── 📋 Task: cf8c61dc-8428-4c7f-b1b1-83fe62128c37
       Status: Executing Task...
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: ca1685aa-4d5c-4111-976f-aea009b17d22
│      Assigned to: Tech Job Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: In Progress
│       └── 🧠 Thinking...
├── 📋 Task: 8c3cc51c-f38a-4977-aa40-bcbe389717b6
│      Assigned to: Personal Profiler for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Tech Job Researcher
│           Status: ✅ Completed
│       ├── 🔧 Used Search the internet with Serper (2)
│       └── 🔧 Used Read website content (2)
├── 📋 Task: d8b2bdeb-4a3a-48a3-9493-d8d194206740
│      Assigned to: Resume Strategist for Engineers
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume Strategist for Engineers
│           Status: ✅ Completed
└── 📋 Task: cf8c61dc-8428-4c7f-b1b1-83fe62128c37
       Assigned to: Engineering Interview Preparer
       Status: ✅ Completed
    └── 🤖 Agent: Engineering Interview Preparer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cf8c61dc-8428-4c7f-b1b1-83fe62128c37                                                                     │
│  Agent: Engineering Interview Preparer                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 91456039-e4fc-440a-b5f3-40880b45ab1d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [49]:
result

CrewOutput(raw="**Interview Questions and Talking Points for Noah**  \n\n1. **Questions about Professional Experience:**\n   - Can you elaborate on the key technology initiatives you led, and how they improved operational efficiency by 30%?  \n     *Talking Point: Discuss specifics of the project, including team dynamics, challenges faced, and innovative strategies employed to achieve the results. Highlight how you fostered collaboration in a remote environment, which is crucial for the AI Fund's work culture.*\n\n2. **Startup Leadership:**\n   - What are some of the biggest challenges you faced while founding and scaling startup ventures, and how did you overcome them?  \n     *Talking Point: Illustrate your entrepreneurial spirit, strategic planning, and ability to adapt in a fast-paced environment. Provide examples of how your leadership style contributed to the startups' growth and success.*\n\n3. **Team Management:**\n   - How do you approach team-building exercises, and what impa

In [39]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

**Noah [Last Name]**  
[Address]  
[City, State, Zip]  
[Phone Number]  
[Email Address]  
[LinkedIn Profile]  

---

**Professional Summary**  
Dynamic and accomplished Software Engineering Leader with over 18 years of robust experience in driving innovation and strategic growth in technology. Proven expertise in managing remote and diverse teams within collaborative environments. Adept at leveraging multiple programming languages and AI methodologies to deliver cutting-edge solutions that align with corporate goals. Passionate about fostering an inclusive and innovative workplace culture, I am well-prepared to contribute significantly to driven organizations like AI Fund that prioritize tech-led transformation.

---

**Core Competencies**  
- Multi-Language Software Development: Python, Java, C++, JavaScript, SQL  
- Leadership & Team Management: Remote & In-Office Teams  
- Artificial Intelligence & Data Science: Deployment of AI Solutions and Predictive Analytics  
- Strategic Innovation: Proven Track Record in Planning & Executing Tech Initiatives  
- Communication & Collaboration: Empathetic Leadership Style Promoting Open Dialogue  

---

**Professional Experience**  

**Tech Initiative Leadership**  
[Company Name | Location | Month Year – Present]  
- Spearheaded technology initiatives that led to a 30% improvement in operational efficiency through strategic project management and resource allocation.  
- Cultivated remote team collaboration enhancing communication workflows resulting in timely project completion.

**Startup Ventures**  
[Company Name | Location | Month Year – Month Year]  
- Founded and scaled startup ventures from concept to full execution, achieving successful product launches within projected timelines.  
- Developed business plans and strategic roadmaps, leading to growth markers exceeding 200% in the first two years.

**Team Management**  
[Company Name | Location | Month Year – Month Year]  
- Implemented team-building exercises that strengthened workplace cohesion and improved project outcomes by 25%.  
- Designed and led training programs for junior engineers, resulting in upskilling and mentorship that fostered talent retention.

---

**Education**  
**Master of Business Administration (MBA)**  
[University Name | Location | Year Graduated]  

**Bachelor of Science in Computer Science**  
[University Name | Location | Year Graduated]  

---

**Projects & Contributions**  
- **AI-Driven Solution Development:** Developed AI models that increased predictive accuracy by 40%, enhancing client decision-making processes.  
- **Community Mentorship:** Actively participated in tech community through workshops and mentorship programs, contributing knowledge and experience to aspiring engineers.  

---

**Interests**  
Enthusiastic about emerging technologies and their transformative potential. Committed to staying ahead of industry trends and fostering community engagement through technology-focused outreach initiatives.

---

**Communication Style**  
Emphasizing clarity, empathy, and transparency, I prioritize open communication to create an environment conducive to innovative teamwork and effective problem-solving.

---

**References**  
Available upon request.  

---

This updated resume effectively highlights Noah’s qualifications and experiences aligned with the job requirements and company culture of AI Fund. It emphasizes strategic leadership, technical expertise, and commitment to fostering an inclusive and collaborative workplace, making him an ideal candidate for further consideration.

In [40]:
display(Markdown("./interview_materials.md"))

**Interview Questions and Talking Points for Noah**  

1. **Questions about Professional Experience:**
   - Can you elaborate on the key technology initiatives you led, and how they improved operational efficiency by 30%?  
     *Talking Point: Discuss specifics of the project, including team dynamics, challenges faced, and innovative strategies employed to achieve the results. Highlight how you fostered collaboration in a remote environment, which is crucial for the AI Fund's work culture.*

2. **Startup Leadership:**
   - What are some of the biggest challenges you faced while founding and scaling startup ventures, and how did you overcome them?  
     *Talking Point: Illustrate your entrepreneurial spirit, strategic planning, and ability to adapt in a fast-paced environment. Provide examples of how your leadership style contributed to the startups' growth and success.*

3. **Team Management:**
   - How do you approach team-building exercises, and what impact have they had on project outcomes in your previous roles?  
     *Talking Point: Share specific exercises and their results, emphasizing your inclusive leadership style and its alignment with AI Fund's values. Highlight how these initiatives led to improved project outcomes and talent retention.*

4. **AI and Data Science Expertise:**
   - Can you describe a successful AI-driven solution you developed and its impact on decision-making for clients?  
     *Talking Point: Detail the process of development, deployment, and measurable benefits of the AI solution. This will showcase your technical proficiency and alignment with AI Fund's focus on leveraging AI for innovation.*

5. **Diversity and Inclusion Principles:**
   - How do your values align with the company’s commitment to fostering diverse and inclusive teams?  
     *Talking Point: Discuss your experience fostering collaboration and empathy within teams, and offer examples of mentorship or outreach initiatives that demonstrate your commitment to diversity in tech.*

6. **Strategic Innovation:**
   - What innovative strategies have you implemented to drive growth and transformation in tech projects?  
     *Talking Point: Highlight specific examples of strategic innovation that led to increased efficiency or growth, connecting your success stories to the mission of AI Fund to drive tech-led transformations.*

7. **Communication and Collaboration:**
   - How do you ensure clear communication within remote and diverse teams?  
     *Talking Point: Discuss your communication style and techniques for nurturing open dialogue, emphasizing the importance of transparency and empathy in achieving effective collaboration.*

8. **Interest in Emerging Technologies:**
   - How do you stay informed about the latest trends and advancements in technology?  
     *Talking Point: Mention your commitment to continuous learning and knowledge sharing within the tech community. Highlight any conferences, workshops, or mentorship programs you are involved in.*

9. **Contribution to Community:**
   - Could you share some insights about your involvement in tech community outreach initiatives?  
     *Talking Point: Describe how this involvement reflects your desire to give back to the tech community and how it aligns with AI Fund’s ethos of fostering growth through collaborative engagement.*

10. **Long-term Vision:**
    - What are your career aspirations in the tech industry, and how do you see yourself contributing to AI Fund specifically?  
      *Talking Point: Use this opportunity to express your enthusiasm for AI Fund's mission and values. Align your vision with the company’s future goals and express how your skills and experiences make you an ideal fit for driving innovation within their team.*

These questions and talking points will give Noah the confidence to articulate his qualifications and experiences effectively, demonstrating how he aligns with the culture and values of AI Fund, ultimately leading to a successful interview.